# 01: Goodfellow GAN Architecture Setup

Purpose: this notebook is the workflow to creating the `GoodfellowGan()` arthictecture for the adversarial package. Note that this is done iteratively between this notebook and within the source package `adversarial/networks.py` - this notebook serves as a mental workthrough for reference and is not intended for experimental runs and may not flow linearly. 

# Imports

In [2]:
from typing import Tuple

import os

import torch
from torch import nn
import torchvision


from adversarial.networks import (GoodfellowG,
                                  GoodfellowD,
                                  GoodfellowGAN)

%load_ext autoreload
%autoreload 2

# Globals

In [3]:
# Directories
EXP_DATA = os.path.abspath(os.path.join("..", "data", "01_raw"))

directories = [EXP_DATA]

# CUDA availability    
DEVICE = torch.device(0) if torch.cuda.is_available() else 'cpu'
print(f"using {DEVICE =}")

using DEVICE ='cpu'


In [4]:
for p in directories:
    if not os.path.exists(p):
        try:
            os.makedirs(p)
        except FileExistsError:
            print("p exists and was somehow uncaught")

# Data

The torchvision MNIST dataset should be just fine. 

In [5]:
img_size = 28
ds_train = torchvision.datasets.MNIST(EXP_DATA, train=True, download=True,
                                transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize((img_size,img_size)),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Lambda((lambda x: torch.flatten(x))),
                            ]))
ds_test = torchvision.datasets.MNIST(EXP_DATA, train=False, download=True,
                                transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize((img_size,img_size)),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Lambda((lambda x: torch.flatten(x))),
                            ]))

In [6]:
ds = torch.utils.data.Subset(ds_train, range(1024))
dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=True)

# Network

In [163]:
img_size = 28**2
latent_dim = 10
hidden_dim = 5
device = DEVICE

g_args = {"img_size": img_size, 
          "latent_dim": latent_dim,
          "hidden_dim": hidden_dim,
          "device": device}

d_args = {"img_size": img_size, 
          "latent_dim": latent_dim,
          "device": device}

g = GoodfellowG(**g_args)
d = GoodfellowD(**d_args)

gf_args = {"G": g,
           "D": d,
           "img_size": img_size, 
           "latent_dim": latent_dim,
           "hidden_dim": hidden_dim,
           "device": device}

gnet = GoodfellowGAN(**gf_args)

In [120]:
def sample_latent_space(latent_dim: int = None, batch_size: int = 32
) -> Tuple[torch.tensor, torch.tensor]:
    """Sample a normal distribution for latent space vectors

    Note, that this assumes row-based data (as expected in Pytorch).

    Args:
        latent_size ): latent space vector, size: batch_size X latent_size

    Returns:
        z, u Tuple[torch.tensor]: batch_size X img_size tensor, and list of zeros as labels
    """

    return torch.randn(batch_size, latent_dim), torch.zeros(batch_size, 1)

# Tests

In [121]:
def test_sample_latent_space(img_size:int=None, batch_size:int=None):
    z, u = sample_latent_space(img_size, batch_size)
    print(z.shape)
    assert z.shape==torch.Size([batch_size, img_size])
    assert all([val==0.0 for val in u])
    
test_sample_latent_space(batch_size=3, img_size=5)
test_sample_latent_space(batch_size=32, img_size=28)

torch.Size([3, 5])
torch.Size([32, 28])


In [10]:
def test_generator_forward(g: GoodfellowG):
    
    input_dim = g.latent_dim
    output_dim = g.img_size
    z, u = g.sample_latent_space(input_dim, batch_size=32)
    
    assert z.shape == (32, input_dim)
    
    z = g.forward(z)
    
    assert z.shape == (32, output_dim)
    
test_generator_forward(g)

# Train Network (sample)

In [115]:
optimizer=torch.optim.Adam(gnet.parameters(), lr=0.001)

In [161]:
outpath = os.path.abspath(os.path.join("..", "data", "05_models"))
if not os.path.exists(outpath):
    os.makedirs(outpath)
    print(f"made {outpath}")

made /Users/u6070354/Desktop/Brian/research/thesis/adv/anti_adv/experiments/goodfellow_gan/data/05_models


In [164]:
gnet.train(dl, optimizer=optimizer, epochs=10, checkpoints=[0,5,9], path=outpath)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0, d_loss: 1.835291862487793


KeyError: -1

In [97]:
dl
x, t = next(iter(dl))
criterion = nn.BCELoss()

print("x shape: ", x.size())
print("t shape: ", t.size())

real_logits = d(x).to(torch.float32)
t = t.to(torch.float32)
print("real logits shape: ", real_logits.size())

x shape:  torch.Size([32, 784])
t shape:  torch.Size([32])
real logits shape:  torch.Size([32, 1])


In [98]:
criterion(real_logits.view(-1, ), t)

tensor(-0.6450, grad_fn=<BinaryCrossEntropyBackward>)

In [85]:
real_logits.view(-1, ).shape

torch.Size([32])

In [70]:
real_logits.shape

torch.Size([32, 1])

In [71]:
d.model

Sequential(
  (0): Linear(in_features=784, out_features=10, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=10, out_features=1, bias=True)
  (7): Sigmoid()
)

In [72]:
x.size()[1]

784